In [1]:
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

%matplotlib inline

dataset = pd.read_csv("weather-data.csv")


# convert 'valid' to datetime format
dataset['valid'] = pd.to_datetime(dataset.valid)

# extract numerical values from datetime object
dataset['year'] = dataset['valid'].dt.year
dataset['month'] = dataset['valid'].dt.month
dataset['week'] = dataset['valid'].dt.week
dataset['day'] = dataset['valid'].dt.day
dataset['hour'] = dataset['valid'].dt.hour
dataset['minute'] = dataset['valid'].dt.minute
dataset['dayofweek'] = dataset['valid'].dt.dayofweek

dataset.head()

#dataset.drop(dataset.columns[[0, 6, 7]], inplace = True) <- This showed me that "station" is actually " station" 
dataset.drop(columns=[' station', 'gust', 'peak_wind_drct', 'valid'], inplace=True)
#dataset.head()
#dataset.info()

#drop na values
dataset.dropna(axis=0, inplace= True)

C:\Users\Boetmate\AppData\Local\Temp\ipykernel_13972\528920820.py:18: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  dataset['week'] = dataset['valid'].dt.week


The RandomForest model will not accept the datetime format, so I had to convert the dates in the 'valid' column into numerical values. 

Since I'm going to be modeling temperature, I can drop all the columns except for my newly created columns (year, month, week, day, hour, minute, dayofweek) and 'tmpc'. 

In [4]:

dataset.info()
dataset.head()

# Define my target: temperature
y = dataset.tmpc 

# Define my x-values, or features
# weather_features = ['year','month', 'day','hour',	'minute','dayofweek']
weather_features = ['year','month', 'day']
X = dataset[weather_features]
X.describe()

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Training and validation sets 
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 1)

# Create Random Forest Model
forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
val_predictions = forest_model.predict(val_X)

#Evaluating with MAE and R2
mean_absolute_error = mean_absolute_error(val_predictions,val_y)

print(f'The mean absolute error is {mean_absolute_error}')

forest_model_score = forest_model.score(val_X, val_y)

print(f'The score is {forest_model_score}')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103584 entries, 0 to 103712
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   tmpc       103584 non-null  float64
 1   dwpc       103584 non-null  float64
 2   relh       103584 non-null  float64
 3   sknt       103584 non-null  float64
 4   year       103584 non-null  int64  
 5   month      103584 non-null  int64  
 6   week       103584 non-null  int64  
 7   day        103584 non-null  int64  
 8   hour       103584 non-null  int64  
 9   minute     103584 non-null  int64  
 10  dayofweek  103584 non-null  int64  
dtypes: float64(4), int64(7)
memory usage: 9.5 MB
The mean absolute error is 1.7549311546515507
The score is 0.7378968915590258


In [5]:
val_X.head()

,year,month,day
25412,2016,6,30
80621,2019,9,6
43607,2017,7,25
85886,2019,12,25
88709,2020,2,21


In [8]:
# Read the csv file with the dates for January 2021
future_df = pd.read_csv("jan_2021.csv")

# Predict the future!! 
future_predictions = forest_model.predict(future_df)

print(future_predictions)

# append future_predictions as a column 

[18.329037   19.83728086 18.15212943 17.46081145 18.33134777 17.31644315
 16.03740394 16.74998331 17.8297585  17.10155477 17.38994476 18.43078227
 18.52319135 17.08369506 19.1726561  19.3927738  18.9254504  19.01419741
 20.24987796 20.74532275 19.53195043 19.30556068 21.66056223 21.36729359
 21.55532283 20.40985739 21.42744282 22.10521631 22.21105655 22.0435968 ]


### Model Evaluation

The forest model score = 0.7379, meaning 73.79% of the variation in temperature is explained by the model.

The mean absolute error tells me that the model's predictions are off by an average of 1.75 degrees Celsius from the true temperature. 

These are reasonably good scores given that the only independent variables in my model are month, day, and year. Ideally, I'd be able to use the other columns from the weather dataset (relative humidity, dew point) to make a more accurate model. However, I can only pass in month, day, and year as variables into my model because I don't know what the relative humidity or dew point would be in January 2021. 